Input: video files


Output: 
  - Frame applied hann window
  - Label of sound or silence in each frame


Parameters:
  - Window length = 30 ms
  - Hop_len / win_length = 1/4

In [1]:
!pip install webrtcvad
import librosa
import pandas as pd
import numpy as np
import librosa.display
import matplotlib.pyplot as plt
from scipy.signal import get_window
import librosa.display
import os
import webrtcvad
import csv

     |████████████████████████████████| 71kB 4.4MB/s 
  Created wheel for webrtcvad: filename=webrtcvad-2.0.10-cp37-cp37m-linux_x86_64.whl size=72340 sha256=a0ce09dc0f907c01855cf57616a2e79b42e0128df13108febafb34bbe12d6258
  Stored in directory: /root/.cache/pip/wheels/44/2a/18/bd1aec41cac7c3051fe95d92a6ed446122ea31dc713c432fa1
Successfully built webrtcvad


In [3]:
def window_frame(y, n_fft, hop_length, win_length, window='hann'):
  # Set default length
  if win_length is None:
      win_length = n_fft
  # Set the default hop, if it's not already specified
  if hop_length is None:
      hop_length = win_length
  fft_window = librosa.filters.get_window(window, win_length, fftbins=True)
  # Pad the window out to n_fft size
  fft_window = librosa.util.pad_center(fft_window, n_fft)
  # Reshape so that the window can be broadcast
  fft_window = fft_window.reshape((-1, 1))
  # Pad the time series so that frames are centered
  y = np.pad(y, int(n_fft // 2), mode='reflect')
  # Window the time series.
  y_frames = librosa.util.frame(y, frame_length=n_fft, hop_length=hop_length)
  windowed_frames = (y_frames * fft_window).T
  return windowed_frames

In [ ]:
path = "/content/train_files"
Y = []
Win_len = []
frame_dur = 0.03
hop_len_per=0.25
col_frames = os.listdir(path)
for frame in col_frames:
  y, sr = librosa.load(path + "/" + str(frame), sr = 16000)
  #y_shape = y.shape[0]
  #duration = y_shape/sr
  win_len = (int)(sr*frame_dur)
  Y.append(y)
  Win_len.append(win_len)

In [ ]:
Frame = []
for y, win_len in zip(Y, Win_len):
  temp_frames = window_frame(y, win_len, int(win_len*hop_len_per), win_len, 'hann')
  Frame.extend(temp_frames)
np.savetxt("hann_frame_environment.csv", Frame, delimiter=",")

In [ ]:
vad = webrtcvad.Vad(3)
Labely3 = []
SR = 16000
for fra in Frame:
    #fr = b'\x00\x00' * int(SR * frame_dur / 1000)
    fr = np.int16(fra * 32768).tobytes() #change float to bytes
    #fr = np.int16(2*100*16).tobytes() #change float to bytes
    is_Speech = vad.is_speech(fr, SR)
    if (is_Speech):
      #print('1')
      Labely3.append(1)
    else:
      #print('0')
      Labely3.append(0)
#np.savetxt("label_y_100files.csv", Labely3, delimiter=",")

In [ ]:
with open('Label_Environment.csv', 'w') as f:
    for i in Labely3:
        f.write('%d \n' % i)

In [ ]:
print("Number of file: ", len(Y))
print("Win len example: ", Win_len[0])
print("Len of Frame: ", len(Frame))
print("Len of label list: ", len(Labely3))